In [15]:
%load_ext autoreload
%autoreload 2

from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner, SklearnCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from sklearn.datasets import load_iris
import pandas as pd
X, y = load_iris( return_X_y=True)
df = pd.DataFrame(X[:,:2])
df['label'] = y
df

,0,1,label
0,5.1,3.5,0
1,4.9,3.0,0
2,4.7,3.2,0
3,4.6,3.1,0
4,5.0,3.6,0
...,...,...,...
145,6.7,3.0,2
146,6.3,2.5,2
147,6.5,3.0,2
148,6.2,3.4,2


In [17]:
from jenga.corruptions.generic import CategoricalShift
from sklearn.model_selection import train_test_split

df['label'] = df['label'].astype(str)
train_df, test_df = train_test_split(df.copy(deep=True), test_size=.5)

df_corrupted = CategoricalShift(column='label', fraction=.5, sampling='MNAR').transform(test_df)
# df_corrupted[df_corrupted['label']!=test_df['label']]
# df_corrupted['label'] = np.nann 

df_corrupted['label_true'] = test_df['label']
df_corrupted['label_corr'] = df_corrupted['label']

df_corrupted

,0,1,label,label_true,label_corr
18,5.7,3.8,0,0,0
24,4.8,3.4,0,0,0
50,7.0,3.2,0,1,0
36,5.5,3.5,2,0,2
1,4.9,3.0,2,0,2
...,...,...,...,...,...
9,4.9,3.1,0,0,0
102,7.1,3.0,1,2,1
118,7.7,2.6,1,2,1
133,6.3,2.8,1,2,1


In [ ]:
sklearn_cleaner = SklearnCleaner(numeric_columns=[0,1], 
                           categorical_columns=['label'],
                                categorical_precision_threshold=.0).fit(train_df)

df_outliers_removed = sklearn_cleaner.remove_outliers(df_corrupted, columns=['label'])
df_outliers_removed[df_outliers_removed['label_true']!=df_outliers_removed['label_corr']]

Classifer for column label reached 0.6499999999999999
Regressor for column 0/lower reached -0.04286843446592453
Regressor for column 0/median reached 0.5924776910430436
Regressor for column 0/upper reached -0.3840952610214328
Regressor for column 1/lower reached -0.25821708724573833
Regressor for column 1/median reached 0.36491226978953273


/Users/felix/code/jenga/yacl_env/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
/Users/felix/code/jenga/yacl_env/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


Regressor for column 1/upper reached 0.04377744196198213
> /Users/felix/code/jenga/jenga/cleaning/cleaners.py(190)remove_outliers()
    188                 for label_idx, label in enumerate(self.predictors[col].classes_):
    189                     import pdb;pdb.set_trace()
--> 190                     above_prec_predictions = self.predictors[col].thresholds[label] <= y_proba[:,label_idx]
    191                     outliers = above_prec_predictions & (df[col] != y_pred)
    192 

ipdb> n
> /Users/felix/code/jenga/jenga/cleaning/cleaners.py(191)remove_outliers()
    189                     import pdb;pdb.set_trace()
    190                     above_prec_predictions = self.predictors[col].thresholds[label] <= y_proba[:,label_idx]
--> 191                     outliers = above_prec_predictions & (df[col] != y_pred)
    192 
    193             if col in self.numeric_columns:

ipdb> n
> /Users/felix/code/jenga/jenga/cleaning/cleaners.py(188)remove_outliers()
    186                 y_pred

In [14]:
df_corrupted['label'] = np.nan
df_imputed = sklearn_cleaner.impute(df_corrupted, columns=['label'])
# df_corrupted['label_predicted']  = sklearn_cleaner.predictors['label'].predict(df_corrupted)
# df_corrupted[df_outliers_removed['label_true']!=df_outliers_removed['label_corr']]
df_imputed

Imputed 75 values in column label


/Users/felix/code/jenga/yacl_env/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


,0,1,label,label_true,label_corr,label_predicted
98,5.1,2.5,1,1,1,1
124,6.7,3.3,2,2,1,2
145,6.7,3.0,1,2,1,1
131,7.9,3.8,2,2,1,2
135,7.7,3.0,2,2,1,2
...,...,...,...,...,...,...
128,6.4,2.8,1,2,1,1
74,6.4,2.9,1,1,1,1
143,6.8,3.2,2,2,1,2
133,6.3,2.8,2,2,1,2


In [6]:
df_outliers_removed_imputed = sklearn_cleaner.impute(df_corrupted, columns=['label'])
df_outliers_removed_imputed[df_outliers_removed['label_true']!=df_outliers_removed['label_corr']]

,0,1,label,label_true,label_corr,label_predicted
124,6.7,3.3,1,2,1,2
145,6.7,3.0,1,2,1,1
131,7.9,3.8,1,2,1,2
135,7.7,3.0,1,2,1,2
107,7.3,2.9,1,2,1,2
120,6.9,3.2,1,2,1,2
129,7.2,3.0,1,2,1,2
97,6.2,2.9,0,1,0,2
60,5.0,2.0,0,1,0,1
125,7.2,3.2,1,2,1,2


In [7]:
sklearn_cleaner.predictors['label'].best_score_

0.7

In [8]:
# cleaner.fit(task.train_data)
# evaluator = AutoCleaningEvaluator(task, cleaner)

In [9]:
# corruptions = []
# for impacted_column in task.numerical_columns + task.categorical_columns:
#     for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
#         if impacted_column in task.numerical_columns:
#             corruptions.append(Scaling(impacted_column, fraction))
#         for missingness in ['MCAR', 'MAR', 'MNAR']:
#             corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
#             corruptions.append(corruption)

In [10]:
# task = OpenMLTask(0, openml_id=42261)
# model = task.fit_baseline_model(task.train_data, task.train_labels)
# sklearn_cleaner = SklearnCleaner(numeric_columns=task.numerical_columns, 
#                            categorical_columns=task.categorical_columns)
# cleaner = AutoGluonCleaner(numeric_columns=task.numerical_columns, 
#                            categorical_columns=task.categorical_columns)

In [11]:
# sklearn_cleaner.fit(task.train_data)
# sklearn_cleaner_evaluator = AutoCleaningEvaluator(task, sklearn_cleaner)
# sklearn_cleaner_results = sklearn_cleaner_evaluator.evaluate(model, 5, *corruptions[:2])